In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [2]:
#read data from conpressed file
vehicle_data = pd.read_csv("vehicle_data.csv.gz",
               compression="gzip")

#strip extra spaces in field
vehicle_data['fuel_type'].str.rstrip().astype(str)
vehicle_data['elec_hybrid']= vehicle_data['fuel_type'].isin(['ELECTRIC','NONE','OTHER'])
vehicle_data

,vin,registration_class,zip,model_year,body_type,fuel_type,elec_hybrid
0,999407G3573,PAS,12866,1976,SUBN,GAS,False
1,9992313,PAS,13316,1924,CONV,GAS,False
2,998867,PAS,11501,1952,2DSD,GAS,False
3,9983692,PAS,10940,1936,2DSD,GAS,False
4,99782010525,PAS,11225,1978,2DSD,GAS,False
...,...,...,...,...,...,...,...
9375975,AAAAAAKE201612240,PAS,14621,1973,2DSD,GAS,False
9375976,,PAS,14120,1941,4DSD,GAS,False
9375977,,SRF,10580,1967,2DSD,GAS,False
9375978,,PAS,14047,1958,CONV,GAS,False


In [17]:
#Seperate Data by fuel_clas
elec_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == True]

std_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == False]

#Rename Columns
elec_df = elec_df.rename(columns={"elec_hybrid": "Electric"})
std_df = std_df.rename(columns={"elec_hybrid": "Standard"})

#group by
elec_df = elec_df.groupby('zip').agg({'vin':'count'})
elec_df = elec_df.fillna(0)

std_df = std_df.groupby('zip').agg({'vin':'count'})
std_df = std_df.fillna(0)

elec_df = elec_df.rename(columns={"vin": "Electric"})
std_df = std_df.rename(columns={"vin": "Standard"})

#Merge the 2 data sets
combined_df = pd.merge(elec_df, std_df, on=['zip'], how="left" )

In [69]:
#Add AQI data
aqi_df = pd.read_csv("AQIndex2020.csv", index_col=1)

#merge with above
type_aqi_df = pd.merge(aqi_df, combined_df, on=['zip'], how="left" )


In [71]:
#merge data based on zip
fuel_aqi_df = pd.merge(aqi_df, combined_df, on=['zip'], how="left" )
fuel_aqi_df = fuel_aqi_df.drop(columns= ['Unnamed: 0','Category Number', 'Category Name','Pollutant' ])
fuel_aqi_df['Percent_Green']= (fuel_aqi_df['Electric']/fuel_aqi_df['Standard'])

#clean Up formatting
fuel_aqi_df['Percent_Green']= fuel_aqi_df['Percent_Green'].astype(float).map("{:.2%}".format)

#Fill na
fuel_aqi_df.head(100)

,AQI,Electric,Standard,Percent_Green
zip,,,,
12023,46,4.0,1510.0,0.26%
12024,46,1.0,128.0,0.78%
12025,46,4.0,4063.0,0.10%
12027,46,15.0,3096.0,0.48%
12028,46,NaN,NaN,nan%
...,...,...,...,...
12074,46,1.0,2339.0,0.04%
12075,46,7.0,2457.0,0.28%
12076,46,1.0,1127.0,0.09%
